**Init Communication**


In [ ]:
import compas_rrc as rrc
import json

from compas.geometry import Point, Vector, Frame, Translation, Rotation

ros = rrc.RosClient()
ros.run()

filma = rrc.AbbClient(ros, "/rob2")
morda = rrc.AbbClient(ros, "/rob1")

connect_filma = filma.send_and_wait(rrc.PrintText("Connecting to Filma..."))
connect_morda = morda.send_and_wait(rrc.PrintText("Connecting to Morda..."))

if connect_filma == "Done" and connect_morda == "Done":
    print("Connected to both robots successfully.")
else:
    print("Failed to connect to one or both robots.")
    exit(1)

def go_home_mortadela():
    return morda.send_and_wait(rrc.MoveToJoints(joints=[-69.24,-12.78,28.21,-1.82,75,-155.83],ext_axes=None,speed=200,zone=rrc.Zone.FINE))

def open_gripper_mordadela():
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO8", 1)) # OPEN
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO6", 0)) # OPEN

def close_gripper_mordadela():
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO8", 0)) # CLOSE
    morda.send_and_wait(rrc.SetDigital("ABB_Scalable_IO_0_DO6", 1)) # CLOSE

In [ ]:

def pick_and_place(robot, pick_frame, place_frame, waypoint_frame, speed, zone):
    transform = Translation.from_vector(pick_frame.zaxis * -150)
    approach_pick = pick_frame.transformed(transform)

    transform = Translation.from_vector(place_frame.zaxis * -150)
    approach_place = place_frame.transformed(transform)
    robot.send_and_wait(rrc.MoveToFrame(frame=approach_pick, speed=speed, zone=rrc.Zone.FINE, motion_type=rrc.Motion.JOINT))
    # open_gripper_mordadela()
    robot.send_and_wait(rrc.MoveToFrame(frame=pick_frame, speed=speed/5, zone=rrc.Zone.FINE, motion_type=rrc.Motion.LINEAR))
    robot.send_and_wait(rrc.WaitTime(1.0))
    # close_gripper_mordadela()

    robot.send_and_wait(rrc.MoveToFrame(frame=approach_pick, speed=speed, zone=zone, motion_type=rrc.Motion.LINEAR))


    robot.send_and_wait(rrc.MoveToFrame(frame=waypoint_frame, speed=speed, zone=zone, motion_type=rrc.Motion.JOINT))

    robot.send_and_wait(rrc.MoveToFrame(frame=approach_place, speed=speed, zone=zone, motion_type=rrc.Motion.JOINT))
    robot.send_and_wait(rrc.MoveToFrame(frame=place_frame, speed=speed/5, zone=zone, motion_type=rrc.Motion.LINEAR))
    robot.send_and_wait(rrc.WaitTime(1.0))
    # open_gripper_mordadela()

    robot.send_and_wait(rrc.MoveToFrame(frame=approach_place, speed=speed, zone=zone, motion_type=rrc.Motion.LINEAR))
    task = go_home_mortadela()
    if task == "Done":
        print("Pick and place operation completed successfully.")
        robot.send_and_wait(rrc.PrintText("Pick and place operation completed."))
    else:
        print("Pick and place operation failed.")





In [ ]:
def load_frames_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    def to_compas_frame(frame_data):
        point = Point(*frame_data['position'])
        xaxis = Vector(*frame_data['x_axis'])
        yaxis = Vector(*frame_data['y_axis'])
        return Frame(point, xaxis, yaxis)

    pick_frames = [to_compas_frame(f) for f in data.get("pick_frames", [])]
    place_frames = [to_compas_frame(f) for f in data.get("place_frames", [])]
    waypoint_frame = to_compas_frame(data["waypoint_frame"])

    return pick_frames, place_frames, waypoint_frame


In [ ]:
pick_frames, place_frames, waypoint_frame = load_frames_from_json(r"D:\IAAC_24-25\MRAC_02\Collaborative_print\demo_rrc\demo_rrc\python\Two robos\task_mortadela.json")

In [ ]:
for pick_frame, place_frame in zip(pick_frames, place_frames):
    pick_and_place(morda, pick_frame, place_frame, waypoint_frame, speed=500, zone=rrc.Zone.FINE)
    